In [1]:
import folium
import geopandas as gpd
from shapely.ops import split

import atlas.eugene as eug

In [2]:
import black
import jupyter_black

jupyter_black.load(
    # lab=False,
    line_length=78,
    # verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

In [3]:
n = eug.get_neighborhood_gdf()

In [4]:
dna_wun = n[
    (n["Neighborhood"] == "Downtown Neighborhood Association")
    | (n["Neighborhood"] == "West University Neighbors")
]
not_dna_wun = n[
    ~(n["Neighborhood"] == "Downtown Neighborhood Association")
    & ~(n["Neighborhood"] == "West University Neighbors")
]

In [5]:
renewal = gpd.read_file(
    "data/city-of-eugene/Urban_Renewal_District_Eugene_1405816958535076811.geojson"
)
city = eug.get_city_gdf()

In [6]:
m = renewal.explore(
    tiles="cartodb positron", color="red", fill=False, tooltip="Name"
)
m = not_dna_wun.explore(
    m=m,
    style_kwds={"fillOpacity": 0.2},
    tooltip="Neighborhood",
    color="lightgreen",
)
m = dna_wun.explore(m=m, color="lightblue", tooltip="Neighborhood")
title = f'<h1 align="center">Downtown, West Universtity, and the Urban Renewal Districts</h1>'
m.get_root().html.add_child(folium.Element(title))
m

This shows only DNA and WUN in blue.

In [7]:
riverfront_not_in_na = (
    renewal[renewal["Name"] == "Riverfront Urban Renewal District"]
    .difference(not_dna_wun.dissolve().geometry.iloc[0])
    .difference(dna_wun.dissolve().geometry.iloc[0])
)
# Trim out on littel bit over by Laurel Hill & Glennwood; reassemble in to a GeoSeries
riverfront = gpd.GeoSeries(
    data=riverfront_not_in_na.geometry.iloc[0].geoms[0]
)

In [16]:
m = renewal.explore(
    tiles="cartodb positron", color="red", fill=False, tooltip="Name"
)
m = not_dna_wun.explore(
    m=m,
    style_kwds={"fillOpacity": 0.2},
    tooltip="Neighborhood",
    color="lightgreen",
)
m = dna_wun.explore(m=m, color="lightblue", tooltip="Neighborhood")
m = riverfront.explore(m=m, color="lightblue", tooltip=False)
title = f'<h1 align="center">Downtown and West Universtity merged with available Riverfront</h1>'
m.get_root().html.add_child(folium.Element(title))
m

This adds all of the Riverfront Urban Renewal district that is not currently part of a neighborhood,
with the exception of the sliver between Fairmount, Laurel Hill Valley, and Glenwood. 

In [9]:
rail = gpd.read_file("data/city-of-eugene/Eugene_Railroads_-_HUB.geojson")

In [ ]:
up = rail[rail["OBJECTID"] == 521].geometry.iloc[0]

north_of_up = gpd.GeoSeries(
    data=split(riverfront.geometry.iloc[0], up).geoms[0]
)

In [13]:
m = renewal.explore(
    tiles="cartodb positron", color="red", fill=False, tooltip="Name"
)
m = not_dna_wun.explore(
    m=m,
    style_kwds={"fillOpacity": 0.2},
    tooltip="Neighborhood",
    color="lightgreen",
)
m = dna_wun.explore(m=m, color="lightblue", tooltip="Neighborhood")
m = north_of_up.explore(m=m, color="lightblue", tooltip=False)
title = f'<h1 align="center">Downtown and West Universtity merged with Riverfront North of Union Pacific</h1>'
m.get_root().html.add_child(folium.Element(title))
m

This excludes portions of the Renewal District that are largely occupied by U of O properties.

In [14]:
millrace = gpd.read_file("data/own/millrace/millrace.geojson").geometry.iloc[
    0
]
west_of_millrace = gpd.GeoSeries(
    data=split(riverfront.geometry.iloc[0], millrace).geoms[0]
)

In [15]:
m = renewal.explore(
    tiles="cartodb positron", color="red", fill=False, tooltip="Name"
)
m = not_dna_wun.explore(
    m=m,
    style_kwds={"fillOpacity": 0.2},
    tooltip="Neighborhood",
    color="lightgreen",
)
m = dna_wun.explore(m=m, color="lightblue", tooltip="Neighborhood")
m = west_of_millrace.explore(m=m, color="lightblue", tooltip=False)
title = f'<h1 align="center">Downtown and West Universtity merged with Riverfront West of Millrace</h1>'
m.get_root().html.add_child(folium.Element(title))
m

This keeps the undevelopted land West of the Millrace, while excluding U of O and park areas.